In [ ]:
from pathlib import Path
import pandas as pd

script_dir = Path().resolve()  # current working directory
file_path = script_dir / "data" / "agents_stats.csv"

df = pd.read_csv(file_path)
pd.set_option('display.max_colwidth', None)
df['map_utilizations'].head(80)


0     {'Lotus': 100.0, 'Bind': 13.0, 'Sunset': 93.0, 'Abyss': 50.0, 'Icebox': 0.0, 'Haven': 100.0, 'Ascent': 88.0}
1       {'Lotus': 0.0, 'Bind': 0.0, 'Sunset': 75.0, 'Abyss': 96.0, 'Icebox': 92.0, 'Haven': 89.0, 'Ascent': 100.0}
2       {'Lotus': 68.0, 'Bind': 90.0, 'Sunset': 0.0, 'Abyss': 21.0, 'Icebox': 100.0, 'Haven': 17.0, 'Ascent': 0.0}
3       {'Lotus': 18.0, 'Bind': 17.0, 'Sunset': 93.0, 'Abyss': 75.0, 'Icebox': 0.0, 'Haven': 78.0, 'Ascent': 38.0}
4          {'Lotus': 75.0, 'Bind': 87.0, 'Sunset': 46.0, 'Abyss': 0.0, 'Icebox': 0.0, 'Haven': 6.0, 'Ascent': 0.0}
5          {'Lotus': 90.0, 'Bind': 63.0, 'Sunset': 11.0, 'Abyss': 4.0, 'Icebox': 0.0, 'Haven': 0.0, 'Ascent': 0.0}
6       {'Lotus': 53.0, 'Bind': 0.0, 'Sunset': 0.0, 'Abyss': 13.0, 'Icebox': 100.0, 'Haven': 22.0, 'Ascent': 50.0}
7         {'Lotus': 8.0, 'Bind': 0.0, 'Sunset': 0.0, 'Abyss': 79.0, 'Icebox': 71.0, 'Haven': 22.0, 'Ascent': 88.0}
8         {'Lotus': 5.0, 'Bind': 73.0, 'Sunset': 14.0, 'Abyss': 29.0, 'Icebox': 

##### agent_stats.csv 
- 26 rows: 1 per agent
- 3 columns: Agent name (unique key), pick rate %, pick rate per map % (nested dict)
- Notes: Pick rate per map % will need flattening before analysis. Keep 0% pick rates though.
- Potential: Can join with other csvs for deeper analysis. 
##### detailed_matches_maps.csv
-  7 columns: match_id, map_name, map_order, score, winner, duration, picked_by.
    - match_id unique for the match, but several maps for each match. One (match_id) to many (map_name). Foreign key probably.
- 86 rows: One per map 
- Notes: Need to figure out how to contextualize "Decider"
- Potential: Look at win rate % by picked_by, see the average map duration for each team to compare against overall success
##### detailed_matches_overview.csv
- 10 columns: match_id, match_title, event, date, format, teams, score, maps_played, patch, pick_ban_info
- 33 rows: One for each match. 
- Notes: match_id can be linked to detailed_matches_maps , match score is easily visible, pick & ban info available. 
- Potential: Use to track what maps people banned & picked (totals) and how they performed on those maps later.
##### detailed_matches_player_stats.csv
- 25 columns
    - Useful columns: match_id, rating, player_id (unique identifier)
- Notes: Large. 1200 rows. Data for each player on each team for each map. 
- Potential: Can be used to track trends over time for each player instead of viewing stats in aggregate.
##### economy_data.csv
- 8 columns: map, team, pistol won, eco (won), semi-eco(won) semi-buy(won) full buy (won) match_id
- 216 rows: One for each team, each map.
- Notes: Need to flatten data format. 1(0) and 2(0) etc. 
- Potential: Identifying trends in types of rounds won
##### event_info.csv
- 6 columns: url, title, subtitle, dates, prize_pool, location.
- 1 row: 1 value for each column.
- No potential for this project outside of basic event info.
##### maps_stats.csv
- 4 columns: map_name, times_played, attack_win_percent, defense_win_percent
- 7 rows: One for each map that was played throughout the tournament.
- Notes: Shows high level stats for map stats.
- Potential:  Easy to see win % by side. 
##### matches.csv
- 13 columns: date, match_id, time, team1, score1, team2, score2, winner, status, week, stage
- 34 rows: One for every match played throughout the tournament
- Notes: Some columns can be dropped (time, status). 
- Potential: Might be helpful to pair with other tables. 
##### performance_data.csv
- 17 columns: Match ID, map, player, team, agent, 2k, 3k, 4k, 5k, 1v1, 1v2, 1v3, 1v4, 1v5, econ, PL, DE
- 860 rows: One for every player for every match. 10 rows per map. 
- Notes: Has agent information, round information, econ rating, plants and defuses. 
- Potential: Identify trends in designated planters. Measure econ rating as an indicator of success.  
##### player_stats.csv
- 25 columns: player, player_name, team, player_id, agents_count, agents, rounds, rating, acs, kd_ratio, kast, adr, kpr, apr, fkpr, fdpr, hs_percent, cl_percent, clutches, k_max, kills, deaths, assists, first_kills, first_deaths
- 80 rows: One for each player in 
- Notes: Need to clean 'agents' column, standardize data types for percentages. 
- Potential: See if certain players were more successful playing more or less agents. Helpful in determining best overall players. 


Now we'll clean the data to get it ready to load into PostgreSQL!

11 columns. 10 useable ones. 

In [ ]:
df = pd.read_csv(file_path)
pd.set_option('display.max_colwidth', None)
df['map_utilizations'].head(80)

# Must convert each key into a column & value into each row. 

0     {'Lotus': 100.0, 'Bind': 13.0, 'Sunset': 93.0, 'Abyss': 50.0, 'Icebox': 0.0, 'Haven': 100.0, 'Ascent': 88.0}
1       {'Lotus': 0.0, 'Bind': 0.0, 'Sunset': 75.0, 'Abyss': 96.0, 'Icebox': 92.0, 'Haven': 89.0, 'Ascent': 100.0}
2       {'Lotus': 68.0, 'Bind': 90.0, 'Sunset': 0.0, 'Abyss': 21.0, 'Icebox': 100.0, 'Haven': 17.0, 'Ascent': 0.0}
3       {'Lotus': 18.0, 'Bind': 17.0, 'Sunset': 93.0, 'Abyss': 75.0, 'Icebox': 0.0, 'Haven': 78.0, 'Ascent': 38.0}
4          {'Lotus': 75.0, 'Bind': 87.0, 'Sunset': 46.0, 'Abyss': 0.0, 'Icebox': 0.0, 'Haven': 6.0, 'Ascent': 0.0}
5          {'Lotus': 90.0, 'Bind': 63.0, 'Sunset': 11.0, 'Abyss': 4.0, 'Icebox': 0.0, 'Haven': 0.0, 'Ascent': 0.0}
6       {'Lotus': 53.0, 'Bind': 0.0, 'Sunset': 0.0, 'Abyss': 13.0, 'Icebox': 100.0, 'Haven': 22.0, 'Ascent': 50.0}
7         {'Lotus': 8.0, 'Bind': 0.0, 'Sunset': 0.0, 'Abyss': 79.0, 'Icebox': 71.0, 'Haven': 22.0, 'Ascent': 88.0}
8         {'Lotus': 5.0, 'Bind': 73.0, 'Sunset': 14.0, 'Abyss': 29.0, 'Icebox': 